In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Reshape, Softmax, Layer, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, ResNet50, EfficientNetB0

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory("data/train",
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical',
                                               subset='training')

val_data = val_datagen.flow_from_directory("data/train",
                                           target_size=(224, 224),
                                           batch_size=32,
                                           class_mode='categorical',
                                           subset='validation')

test_data = test_datagen.flow_from_directory("data/test",
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical',
                                             shuffle=False)

input_shape = (224, 224, 3)
num_classes = train_data.num_classes

Found 4187 images belonging to 2 classes.
Found 1045 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [11]:
#CNN

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
131/131 [==============================] - 56s 398ms/step - loss: 0.5725 - accuracy: 0.8584 - val_loss: 14.2011 - val_accuracy: 0.7426
Epoch 2/10
131/131 [==============================] - 47s 361ms/step - loss: 0.2208 - accuracy: 0.9054 - val_loss: 20.7720 - val_accuracy: 0.7426
Epoch 3/10
131/131 [==============================] - 47s 360ms/step - loss: 0.2075 - accuracy: 0.9188 - val_loss: 6.7399 - val_accuracy: 0.7426
Epoch 4/10
131/131 [==============================] - 48s 362ms/step - loss: 0.1840 - accuracy: 0.9245 - val_loss: 1.9244 - val_accuracy: 0.7435
Epoch 5/10
131/131 [==============================] - 49s 373ms/step - loss: 0.1785 - accuracy: 0.9245 - val_loss: 0.2085 - val_accuracy: 0.9254
Epoch 6/10
131/131 [==============================] - 49s 370ms/step - loss: 0.1642 - accuracy: 0.9353 - val_loss: 0.2218 - val_accuracy: 0.9043
Epoch 7/10
131/131 [==============================] - 49s 371ms/step - loss: 0.1625 - accuracy: 0.9334 - val_loss: 0.1725 - val_

In [4]:
#Hybrid

class SelfAttention(Layer):
    def __init__(self, embed_dim):
        super(SelfAttention, self).__init__()
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.softmax = Softmax(axis=-1)

    def call(self, inputs):
        Q = self.query_dense(inputs)
        K = self.key_dense(inputs)
        V = self.value_dense(inputs)

        attention_scores = tf.matmul(Q, K, transpose_b=True)
        attention_weights = self.softmax(attention_scores)

        return tf.matmul(attention_weights, V)
    
def build_hybrid_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(128, activation="relu")(x)
    x = Reshape((1, 128))(x)

    x = SelfAttention(128)(x)
    x = Flatten()(x)

    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)
    return model

model = build_hybrid_model(input_shape, num_classes)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
131/131 [==============================] - 61s 420ms/step - loss: 0.5311 - accuracy: 0.7413 - val_loss: 0.5951 - val_accuracy: 0.7426
Epoch 2/10
131/131 [==============================] - 49s 374ms/step - loss: 0.4083 - accuracy: 0.7839 - val_loss: 0.4604 - val_accuracy: 0.7694
Epoch 3/10
131/131 [==============================] - 49s 373ms/step - loss: 0.3779 - accuracy: 0.8132 - val_loss: 0.4128 - val_accuracy: 0.7885
Epoch 4/10
131/131 [==============================] - 49s 374ms/step - loss: 0.3429 - accuracy: 0.8376 - val_loss: 0.3772 - val_accuracy: 0.8239
Epoch 5/10
131/131 [==============================] - 50s 379ms/step - loss: 0.3250 - accuracy: 0.8486 - val_loss: 0.3683 - val_accuracy: 0.8268
Epoch 6/10
131/131 [==============================] - 50s 377ms/step - loss: 0.3297 - accuracy: 0.8517 - val_loss: 0.3248 - val_accuracy: 0.8555
Epoch 7/10
131/131 [==============================] - 49s 373ms/step - loss: 0.3147 - accuracy: 0.8534 - val_loss: 0.3631 - val_ac

In [ ]:
#DenseNet121

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:-31]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)    

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
131/131 [==============================] - 54s 379ms/step - loss: 0.2433 - accuracy: 0.8942 - val_loss: 0.1364 - val_accuracy: 0.9483
Epoch 2/10
131/131 [==============================] - 48s 368ms/step - loss: 0.1363 - accuracy: 0.9467 - val_loss: 0.0873 - val_accuracy: 0.9656
Epoch 3/10
131/131 [==============================] - 49s 370ms/step - loss: 0.0970 - accuracy: 0.9630 - val_loss: 0.0993 - val_accuracy: 0.9541
Epoch 4/10
131/131 [==============================] - 48s 368ms/step - loss: 0.0839 - accuracy: 0.9713 - val_loss: 0.0965 - val_accuracy: 0.9589
Epoch 5/10
131/131 [==============================] - 48s 368ms/step - loss: 0.0777 - accuracy: 0.9718 - val_loss: 0.0570 - val_accuracy: 0.9732
Epoch 6/10
131/131 [==============================] - 48s 369ms/step - loss: 0.0720 - accuracy: 0.9733 - val_loss: 0.0606 - val_accuracy: 0.9761
Epoch 7/10
131/131 [==============================] - 48s 369ms/step - loss: 0.0606 - accuracy: 0.9785 - val_loss: 0.0696 - val_ac

In [6]:
#ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:140]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
131/131 [==============================] - 53s 376ms/step - loss: 0.3174 - accuracy: 0.8538 - val_loss: 0.4969 - val_accuracy: 0.7426
Epoch 2/10
131/131 [==============================] - 48s 366ms/step - loss: 0.2405 - accuracy: 0.8937 - val_loss: 2.0372 - val_accuracy: 0.7445
Epoch 3/10
131/131 [==============================] - 48s 363ms/step - loss: 0.2345 - accuracy: 0.8983 - val_loss: 1.0212 - val_accuracy: 0.7579
Epoch 4/10
131/131 [==============================] - 48s 363ms/step - loss: 0.2093 - accuracy: 0.9123 - val_loss: 1.6745 - val_accuracy: 0.7589
Epoch 5/10
131/131 [==============================] - 48s 364ms/step - loss: 0.2118 - accuracy: 0.9131 - val_loss: 1.1108 - val_accuracy: 0.7493
Epoch 6/10
131/131 [==============================] - 48s 364ms/step - loss: 0.2214 - accuracy: 0.8987 - val_loss: 5.2311 - val_accuracy: 0.7426
Epoch 7/10
131/131 [==============================] - 48s 364ms/step - loss: 0.1913 - accuracy: 0.9178 - val_loss: 1.8567 - val_ac

In [7]:
#EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
131/131 [==============================] - 53s 371ms/step - loss: 0.6009 - accuracy: 0.7311 - val_loss: 0.5819 - val_accuracy: 0.7426
Epoch 2/10
131/131 [==============================] - 48s 364ms/step - loss: 0.5828 - accuracy: 0.7397 - val_loss: 0.5696 - val_accuracy: 0.7426
Epoch 3/10
131/131 [==============================] - 48s 365ms/step - loss: 0.5831 - accuracy: 0.7397 - val_loss: 0.5704 - val_accuracy: 0.7426
Epoch 4/10
131/131 [==============================] - 48s 364ms/step - loss: 0.5813 - accuracy: 0.7423 - val_loss: 0.5727 - val_accuracy: 0.7426
Epoch 5/10
131/131 [==============================] - 48s 365ms/step - loss: 0.5774 - accuracy: 0.7423 - val_loss: 0.5725 - val_accuracy: 0.7426
Epoch 6/10
131/131 [==============================] - 48s 363ms/step - loss: 0.5761 - accuracy: 0.7421 - val_loss: 0.5687 - val_accuracy: 0.7426
Epoch 7/10
131/131 [==============================] - 48s 364ms/step - loss: 0.5681 - accuracy: 0.7423 - val_loss: 0.5923 - val_ac